In [1]:
%%capture
!pip install geedim  # the needed dependencies
!pip install geojson
!pip install geemap
!pip install rasterio

In [2]:
!pip install earthengine-api
!pip install folium


In [5]:
!conda install rasterio -y

NVIDIA: no NVIDIA devices found
Channels:
 - conda-forge
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/vfourel/miniforge3

  added / updated specs:
    - rasterio


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    affine-2.4.0               |     pyhd8ed1ab_1          19 KB  conda-forge
    blosc-1.21.6               |       hef167b5_0          48 KB  conda-forge
    click-plugins-1.1.1        |     pyhd8ed1ab_1          12 KB  conda-forge
    cligj-0.7.2                |     pyhd8ed1ab_2          12 KB  conda-forge
    ffmpeg-2.8.6               |                0        55.0 MB  conda-forge
    freexl-2.0.0               |       h9dce30a_2          58 KB  conda-forge
    geos-3.13.0                |       h5888daf_0         1.8 MB  conda-forge
    geotiff-1.7.3              |       h77b800c_3         128 KB  conda-forge
    json-c-0.18  

In [9]:
import os.path
import zipfile
import os # we import hte necessary packages

# Import necessary modules
import numpy as np
# here we want to plot the data we received
import matplotlib.pyplot as plt
import copy


In [11]:
!conda install -c https://conda.anaconda.org/ioos rasterio


NVIDIA: no NVIDIA devices found
Channels:
 - ioos
 - conda-forge
Platform: linux-64
Solving environment: done

# All requested packages already installed.



In [14]:
import rasterio

ModuleNotFoundError: No module named 'rasterio'

In [ ]:
path = '/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Get the raster equivalent

In [38]:
import re
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt
import rasterio
from rasterio.warp import transform

In [39]:

import pandas as pd

# Load the dataframe
df_sample = pd.read_csv('/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data/Coordinates1Mil/coordinates_Bavaria_1mil.csv')


In [40]:
df_sample

,Longitude,Latitude
0,9.546802,47.866430
1,10.576114,50.424199
2,12.870875,47.267871
3,12.008238,47.817804
4,9.894532,48.561883
...,...,...
149995,12.726839,49.272750
149996,10.892782,49.599303
149997,13.307770,49.339694
149998,12.916161,48.165247


In [41]:
MODIS_NPP_Path_yearly = '/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data/RasterBandsData/YearlyValue/MODIS_NPP'
Elevation_Path_static = '/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data/RasterBandsData/StaticValue/Elevation'
LST_Path_yearly = '/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data/RasterBandsData/YearlyValue/LST'
LAI_Path_yearly = '/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data/RasterBandsData/YearlyValue/LAI'
SoilEvaporation_Path_yearly = '/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data/RasterBandsData/YearlyValue/SoilEvaporation'
TotalEvaporation_Path_yearly = '/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data/RasterBandsData/YearlyValue/TotalEvapotranspiration'
LAI_Path_Season = '/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data/RasterBandsData/SeasonalValue/LAI'
LST_Path_Season = '/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data/RasterBandsData/SeasonalValue/LST'
SoilEvaporation_Path_Season = '/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data/RasterBandsData/SeasonalValue/SoilEvaporation'
TotalEvaporation_Path_Season = '/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data/RasterBandsData/SeasonalValue/TotalEvapotranspiration'


In [42]:
data_paths = [
    '/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data/RasterBandsData/YearlyValue/MODIS_NPP',
    '/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data/RasterBandsData/StaticValue/Elevation',
    '/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data/RasterBandsData/YearlyValue/LST',
    '/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data/RasterBandsData/YearlyValue/LAI',
    '/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data/RasterBandsData/YearlyValue/SoilEvaporation',
    '/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data/RasterBandsData/YearlyValue/TotalEvapotranspiration',
    '/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data/RasterBandsData/SeasonalValue/LAI',
    '/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data/RasterBandsData/SeasonalValue/LST',
    '/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data/RasterBandsData/SeasonalValue/SoilEvaporation',
    '/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data/RasterBandsData/SeasonalValue/TotalEvapotranspiration'
]


In [43]:

def build_kdtree_from_npy_file(file_path):
    # Load data from .npy file
    data = np.load(file_path)

    # Extract longitude and latitude columns
    longitude = data[:, 1]
    latitude = data[:, 2]

    # Build KD-tree
    tree = cKDTree(np.column_stack((longitude, latitude)))

    return tree

def find_closest_indices(tree, query_point, k=4):
    # Query the tree for k nearest neighbors
    distances, indices = tree.query(query_point, k=k)

    return indices

In [44]:

def generate_filename_dict(folder_path):
    filename_dict = {}

    # Iterate over the files in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Extract the ID number from the filename
        match = re.search(r'ID(\d+)N', filename)
        if match:
            id_number = int(match.group(1))

            # Store the filename in the dictionary with the ID number as the key
            filename_dict[id_number] = file_path

    return filename_dict

In [45]:
file_dict = {}
crs_dict = {}

for path in data_paths:
  for root, dirs, files in os.walk(path):
        for dir in dirs:
            dir_path = os.path.join(root, path)
            for file in os.listdir(dir_path):
                if file.endswith('.npy') and dir in file:

                    subfolder = os.path.join(path,dir)
                                        # Get a list of all files in the subfolder
                    files = os.listdir(subfolder)

                    # Sort the list of files
                    files.sort()

                    # Get the first file
                    first_file = files[0]

                    # If you need the full path to the file
                    first_file_path = os.path.join(subfolder, first_file)
                    src_dataset = rasterio.open(first_file_path)
                    # Deep copy the source CRS
                    src_crs = copy.deepcopy(src_dataset.crs)
                    # Close the source dataset
                    src_dataset.close()
                        # Iterate over the files in the folder
                    transform_dict = {}
                    for filename in os.listdir(subfolder):
                        file_path = os.path.join(subfolder, filename)

                        # Extract the ID number from the filename
                        match = re.search(r'ID(\d+)N', filename)
                        if match:
                            id_number = int(match.group(1))

                            # Read the rasterio src.transform
                            with rasterio.open(file_path) as src:
                                src_transform_copy = copy.deepcopy(src.transform)

                            # Store the transform in the dictionary with the ID number as the key
                            transform_dict[id_number] = src_transform_copy
                    file_dict[subfolder] = [ build_kdtree_from_npy_file(os.path.join(dir_path,file)), os.path.join(path,file),generate_filename_dict(subfolder),src_crs,transform_dict]



print(file_dict)
print(crs_dict)

{'/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2001': [<scipy.spatial._ckdtree.cKDTree object at 0x7ac3b9136f10>, '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/bounds_array_MODIS_NPP_2001.npy', {0: '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2001/ID0N48_51114628138105S46_71027383528823W8_981031902268322E10_781904348361152.tif', 1: '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2001/ID1N48_51114628138105S46_71027383528823W10_777662470507366E12_578534916600194.tif', 2: '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2001/ID2N48_51114628138105S46_71027383528823W12_574293038746408E14_375165484839236.tif', 3: '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2001/ID3N50_3077768496201S48_50690440352727W8_981031902

In [46]:
path

'/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/SeasonalValue/TotalEvapotranspiration'

In [47]:
subfolder = Elevation_Path_static
# Get a list of all files in the subfolder
files = os.listdir(subfolder)

                    # Sort the list of files
files.sort()

                    # Get the first file
first_file = files[0]

                    # If you need the full path to the file
first_file_path = os.path.join(subfolder, first_file)
src_dataset = rasterio.open(first_file_path)
                    # Deep copy the source CRS
src_crs = copy.deepcopy(src_dataset.crs)
                    # Close the source dataset
src_dataset.close()
                        # Iterate over the files in the folder
transform_dict = {}
for filename in os.listdir(subfolder):
  file_path = os.path.join(subfolder, filename)
                   # Extract the ID number from the filename
  match = re.search(r'ID(\d+)N', filename)
  if match:
    id_number = int(match.group(1))

                              # Read the rasterio src.transform
  with rasterio.open(file_path) as src:
    src_transform_copy = copy.deepcopy(src.transform)

                              # Store the transform in the dictionary with the ID number as the key
    transform_dict[id_number] = src_transform_copy
file_dict[subfolder] = [ build_kdtree_from_npy_file(os.path.join('/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data/RasterBandsData/StaticValue/','bounds_array_Elevation.npy')), os.path.join('/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/StaticValue/','bounds_array_Elevation.npy'),generate_filename_dict(subfolder),src_crs,transform_dict]


In [48]:

def generate_filename_dict(folder_path):
    filename_dict = {}

    # Iterate over the files in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Extract the ID number from the filename
        match = re.search(r'ID(\d+)N', filename)
        if match:
            id_number = int(match.group(1))

            # Store the filename in the dictionary with the ID number as the key
            filename_dict[id_number] = file_path

    return filename_dict

In [49]:
def transform_coordinates(lon, lat, crs):
    # Coordinates in the target CRS
    x, y = transform({'init': 'EPSG:4326'}, crs, [lon], [lat])
    return x[0], y[0]

In [50]:

def get_tif_ID( latitude, longitude,kdTree,DataBoundsArray,DictFilenames,src_crs):
    x, y = transform_coordinates(longitude, latitude,src_crs)
    index_list = find_closest_indices(kdTree,[y,x])
    DataBoundsArray = np.load(DataBoundsArray)
    id_list = DataBoundsArray[index_list, 0]

    for id in id_list:
        filename = DictFilenames[id]
        # Replace underscores with periods
        filenameTransformed = filename.replace('_', '.')
        # Open the raster file using rasterio
        #     # Read the raster data (assuming only one band)
        #                     # Check if all data points have the same value
        #     if (raster_data == raster_data[0]).all():
        #       print('All the same value for ',filename)
        #     else:
              # Extract the four floats using regular expressions
        coordinates = re.findall(r'[NSWE]-?\d+\.\d+|[NSWE]0', filenameTransformed)
              # Convert the strings to floats
        coordinates = [float(coord[1:]) for coord in coordinates]
              # coordinates = [0.0 if coord.endswith('0') else float(coord[1:]) if coord != 'E0' else 0.0 for coord in coordinates]

              # Check if the sampled coordinates match the filename
        if coordinates[3] >= x and coordinates[2]  <= x and coordinates[0] >= y and coordinates[1] <= y:
          return x,y, id
        # else:
        #   return 0,0,0
           # print(coordinates,x,y,id)

    #print('f')
    #print(latitude, longitude)
    return 0,0,0

def get_tif_ArrayPosition( latitude, longitude,kdTree,DataBoundsArray,DictFilenames,src_crs,DictTransform):

  x_original,y_original, id = get_tif_ID( latitude, longitude,kdTree,DataBoundsArray,DictFilenames,src_crs)
  x, y = rasterio.transform.rowcol(DictTransform[id], x_original, y_original)
  if x < 0:
    print(x,y,id)
  if y < 0:
    print(y,x,id)
  return latitude, longitude , id, x , y


In [51]:
# Specify the path
path = '/home/vfourel/MasterThesis/Data_Dec2024/DataVersion1Mil/Data/Coordinates1Mil'


In [52]:
df_sample.head(3)

,Longitude,Latitude
0,9.546802,47.866430
1,10.576114,50.424199
2,12.870875,47.267871


In [ ]:
file_dict.keys()

dict_keys(['/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2001', '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2002', '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2003', '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2004', '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2005', '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2006', '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2007', '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2008', '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2009', '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyVa

In [ ]:
# Assume file_dict.keys() contains a list of paths
start_key = 'SeasonalValue/LAI/2003_autumn'
done = 0
start_processing = False

In [ ]:
for subfolder in file_dict.keys():
  # If the start key is found, enable processing
  # Extract the last 3 parts of the path
  last_three_parts = '/'.join(subfolder.split(os.sep)[-3:])

  if last_three_parts == start_key:
      start_processing = True
      print(done)
      continue  # Skip the start key itself
  # Only process keys after the start key
  if start_processing:
    samplingOn = np.array(list(map(lambda coord: get_tif_ArrayPosition(float(coord[1]), float(coord[0]), file_dict[subfolder][0] , file_dict[subfolder][1] , file_dict[subfolder][2] , file_dict[subfolder][3] , file_dict[subfolder][4]), df_sample.to_numpy())))
    # Get the last three subfolders in the path
    subfolders = subfolder.split(os.sep)[-3:]

    # Create subfolders within the path
    new_path = os.path.join(path, *subfolders)

    # Check if the new path exists, if not, create it
    if not os.path.exists(new_path):
      os.makedirs(new_path)
    print(new_path)
    # Specify the file name
    filename = 'coordinates.npy'

    # Specify the full path to the file
    file_path = os.path.join(new_path, filename)

    # Save the numpy matrix
    np.save(file_path, samplingOn)
  done += 1


133
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/Coordinates/SeasonalValue/LAI/2004_winter
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/Coordinates/SeasonalValue/LAI/2004_spring


# Yearly: get OC Coordinates

In [53]:
import pandas as pd

# Specify the path to the Excel file
filename = '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/LUCAS_LFU_Lfl_00to23_Bavaria_OC.xlsx'

# Read the 'GPS_LONG', 'GPS_LAT', and 'year' columns
df = pd.read_excel(filename, usecols=['GPS_LONG', 'GPS_LAT', 'year'])

print(df)


        GPS_LONG    GPS_LAT  year
0      10.429133  49.398176  2001
1      10.429133  49.398176  2001
2      10.429133  49.398176  2001
3      10.429133  49.398176  2001
4      10.429133  49.398176  2001
...          ...        ...   ...
30446  12.047354  50.579570  2009
30447  11.677810  50.513615  2009
30448  10.408938  50.525138  2009
30449  10.098825  50.579766  2009
30450  10.548066  50.362728  2009

[30451 rows x 3 columns]


In [ ]:
pathLUCAS = '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/LUCAS_LFU_Lfl_00to23_Bavaria_OC.xlsx'

In [ ]:
LUCAScoordinates = "/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/OC_LUCAS_LFU_LfL_Coordinates"

In [54]:
file_dict.keys()

dict_keys(['/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2001', '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2002', '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2003', '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2004', '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2005', '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2006', '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2007', '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2008', '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2009', '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyVa

In [ ]:

for subfolder in file_dict.keys():
    # Get the last subfolder in the path
    last_subfolder = subfolder.split(os.sep)[-1]
    print(subfolder)
    subfolders = subfolder.split(os.sep)[-3:]

    # Try to convert the last subfolder to an integer (assuming the year is represented as an integer)
    try:
        year = int(last_subfolder)
    except ValueError:
        continue  # If it can't be converted to an integer, skip this iteration of the loop

    # Filter the DataFrame based on the year
    df_year = df[df['year'] == year]
    # Remove rows where 'GPS_LONG' or 'GPS_LAT' are not finite numbers
    df_year = df_year[np.isfinite(df_year['GPS_LONG']) & np.isfinite(df_year['GPS_LAT'])]
    # If the DataFrame is empty, skip this iteration of the loop
    if df_year.empty:
        continue
    samplingOn = np.array(list(map(lambda coord: get_tif_ArrayPosition(float(coord[1]), float(coord[0]), file_dict[subfolder][0] , file_dict[subfolder][1] , file_dict[subfolder][2] , file_dict[subfolder][3] , file_dict[subfolder][4]), df_year.to_numpy())))
    # Get the last three subfolders in the path

    # Create subfolders within the path
    new_path = os.path.join(LUCAScoordinates, *subfolders)

    #  Check if the new path exists, if not, create it
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    print(new_path)
    # Specify the file name
    filename = 'coordinates.npy'

    # Specify the full path to the file
    file_path = os.path.join(new_path, filename)

    # Save the numpy matrix
    np.save(file_path, samplingOn)


/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2001
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/OC_LUCAS_LFU_LfL_Coordinates/YearlyValue/MODIS_NPP/2001
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2002
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/OC_LUCAS_LFU_LfL_Coordinates/YearlyValue/MODIS_NPP/2002
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2003
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/OC_LUCAS_LFU_LfL_Coordinates/YearlyValue/MODIS_NPP/2003
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2004
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/OC_LUCAS_LFU_LfL_Coordinates/YearlyValue/MODIS_NPP/2004
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2005
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/OC_LUCAS_LF

In [55]:

    # Get the last subfolder in the path
last_subfolder = Elevation_Path_static.split(os.sep)[-1]
subfolders = Elevation_Path_static.split(os.sep)[-3:]
last_subfolder2 = Elevation_Path_static.split(os.sep)[-2]

    # Try to convert the last subfolder to an integer (assuming the year is represented as an integer)
     # If it can't be converted to an integer, skip this iteration of the loop

    # Filter the DataFrame based on the year
df_year = df
    # Remove rows where 'GPS_LONG' or 'GPS_LAT' are not finite numbers
df_year = df_year[np.isfinite(df_year['GPS_LONG']) & np.isfinite(df_year['GPS_LAT'])]
    # If the DataFrame is empty, skip this iteration of the loop

samplingOn = np.array(list(map(lambda coord: get_tif_ArrayPosition(float(coord[1]), float(coord[0]), file_dict[Elevation_Path_static][0] , file_dict[Elevation_Path_static][1] , file_dict[Elevation_Path_static][2] , file_dict[Elevation_Path_static][3] , file_dict[Elevation_Path_static][4]), df_year.to_numpy())))
    # Get the last three subfolders in the path

    # Create subfolders within the path
new_path = os.path.join(LUCAScoordinates, *subfolders)

    #  Check if the new path exists, if not, create it
if not os.path.exists(new_path):
  os.makedirs(new_path)
print(new_path)
    # Specify the file name
filename = 'coordinates.npy'



/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/OC_LUCAS_LFU_LfL_Coordinates/RasterBandsData/StaticValue/Elevation


In [56]:
# Specify the full path to the file
file_path = os.path.join(new_path, filename)

    # Save the numpy matrix
np.save(file_path, samplingOn)

In [ ]:
len(samplingOn)

63

# Season: get OC Coordinates

In [ ]:
pathSeason = "/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/LUCAS_LFU_Bavaria_OC_joint_data_modified.xlsx"

In [ ]:
import pandas as pd

# Specify the path to the Excel file
filename = pathSeason
# Read the 'GPS_LONG', 'GPS_LAT', and 'survey_date' columns
df = pd.read_excel(filename, usecols=['GPS_LONG', 'GPS_LAT', 'survey_date'])

# Convert 'survey_date' to datetime format
df['survey_date'] = pd.to_datetime(df['survey_date'])

# Extract the year and month from 'survey_date'
df['year'] = df['survey_date'].dt.year
df['month'] = df['survey_date'].dt.month

# Define the months for each season
seasons = {
    'winter': [12, 1, 2],
    'spring': [3, 4, 5],
    'summer': [6, 7, 8],
    'autumn': [9, 10, 11]
}

# Adjust the logic for assigning seasons:
def get_season_and_adjusted_year(month, year):
    if month == 12:  # December should be part of next year's winter
        return 'winter', year + 1
    elif month in [1, 2]:
        return 'winter', year
    elif month in [3, 4, 5]:
        return 'spring', year
    elif month in [6, 7, 8]:
        return 'summer', year
    elif month in [9, 10, 11]:
        return 'autumn', year

# Apply the function to get 'season' and adjusted 'year'
df['season_year'] = df.apply(lambda row: get_season_and_adjusted_year(row['month'], row['year']), axis=1)

# Separate the tuple into 'season' and 'year'
df['season'] = df['season_year'].apply(lambda x: x[0])
df['adjusted_year'] = df['season_year'].apply(lambda x: x[1])

# Create a 'year_season' column using the adjusted year and the season
df['year_season'] = df['adjusted_year'].astype(str) + '_' + df['season']

# Create a list of years (2000 to 2023)
years = list(range(2000, 2024))  # This will create a list from 2000 to 2023

# Filter the DataFrame based on the year and season
df_filtered = df[df['adjusted_year'].isin(years) & df['season'].isin(seasons.keys())]

print(df_filtered)



<ipython-input-30-d3f1ab0f309c>:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['survey_date'] = pd.to_datetime(df['survey_date'])


       GPS_LONG    GPS_LAT survey_date  year  month     season_year  season  \
0     12.643109  48.214171  2000-09-20  2000      9  (autumn, 2000)  autumn   
1     12.807782  48.213941  2000-09-21  2000      9  (autumn, 2000)  autumn   
2     12.529378  48.067979  2000-09-15  2000      9  (autumn, 2000)  autumn   
3     12.758653  47.928370  2000-09-14  2000      9  (autumn, 2000)  autumn   
4     12.829474  47.562610  2000-01-08  2000      1  (winter, 2000)  winter   
...         ...        ...         ...   ...    ...             ...     ...   
9918  12.047354  50.579570  2009-05-31  2009      5  (spring, 2009)  spring   
9919  11.677810  50.513615  2009-05-20  2009      5  (spring, 2009)  spring   
9920  10.408938  50.525138  2009-06-20  2009      6  (summer, 2009)  summer   
9921  10.098825  50.579766  2009-06-22  2009      6  (summer, 2009)  summer   
9922  10.548066  50.362728  2009-06-16  2009      6  (summer, 2009)  summer   

      adjusted_year  year_season  
0              2

In [ ]:

for subfolder in file_dict.keys():
    # Get the last subfolder in the path
    last_subfolder = subfolder.split(os.sep)[-1]
    subfolders = subfolder.split(os.sep)[-3:]
    # Try to convert the last subfolder to an integer (assuming the year is represented as an integer)
    try:
        year_season = last_subfolder
    except ValueError:
        continue  # If it can't be converted to an integer, skip this iteration of the loop

    # Filter the DataFrame based on the year
    df_year_season = df[df['year_season'] == year_season]
    # Remove rows where 'GPS_LONG' or 'GPS_LAT' are not finite numbers
    df_year_season = df_year_season[np.isfinite(df_year_season['GPS_LONG']) & np.isfinite(df_year_season['GPS_LAT'])]
    # If the DataFrame is empty, skip this iteration of the loop
    if df_year_season.empty:
        continue
    samplingOn = np.array(list(map(lambda coord: get_tif_ArrayPosition(float(coord[1]), float(coord[0]), file_dict[subfolder][0] , file_dict[subfolder][1] , file_dict[subfolder][2] , file_dict[subfolder][3] , file_dict[subfolder][4]), df_year_season.to_numpy())))
    # Get the last three subfolders in the path

    # Create subfolders within the path
    new_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/OC_LUCAS_LFU_LfL_Coordinates', *subfolders)

    #  Check if the new path exists, if not, create it
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    print(new_path)
    # Specify the file name
    filename = 'coordinates.npy'

    # Specify the full path to the file
    file_path = os.path.join(new_path, filename)

    # Save the numpy matrix
    np.save(file_path, samplingOn)


/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/OC_LUCAS_LFU_LfL_Coordinates/SeasonalValue/LAI/2000_winter
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/OC_LUCAS_LFU_LfL_Coordinates/SeasonalValue/LAI/2000_spring
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/OC_LUCAS_LFU_LfL_Coordinates/SeasonalValue/LAI/2000_summer
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/OC_LUCAS_LFU_LfL_Coordinates/SeasonalValue/LAI/2000_autumn
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/OC_LUCAS_LFU_LfL_Coordinates/SeasonalValue/LAI/2001_winter
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/OC_LUCAS_LFU_LfL_Coordinates/SeasonalValue/LAI/2001_spring
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/OC_LUCAS_LFU_LfL_Coordinates/SeasonalValue/LAI/2001_summer
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/OC_LUCAS_LFU_LfL_Coordinates/SeasonalValue/LAI/2001_autumn
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/OC_LUCAS_LFU_LfL_Coordinates/Seas